# Milestone 3: Memory Integration with LangGraph

This notebook demonstrates conversation memory integration:
1. Create conversation memory
2. Integrate memory with RAG system
3. Test multi-turn conversations
4. Implement LangGraph-based chat agent

## Setup

In [ ]:
# Import required modules
import sys
sys.path.append('..')

from src import memory, rag_baseline
from dotenv import load_dotenv

load_dotenv()
print("Memory module loaded successfully!")

## Step 1: Create Conversation Memory

Initialize a conversation memory buffer.

In [ ]:
# Create conversation memory
conv_memory = memory.ConversationMemory(max_history=10)

print("Conversation memory created")
print(f"Max history: {conv_memory.max_history} turns")
print(f"Current history: {len(conv_memory.history)} messages")

## Step 2: Test Memory Operations

Add conversation turns and retrieve history.

In [ ]:
# Add conversation turns
conv_memory.add_turn(
    "What is GDPR?",
    "GDPR is the General Data Protection Regulation..."
)

conv_memory.add_turn(
    "When did it come into effect?",
    "It came into effect on May 25, 2018."
)

conv_memory.add_turn(
    "What are the key principles?",
    "The key principles include lawfulness, fairness, and transparency..."
)

# Display history
print("\nConversation History:")
print("=" * 60)
print(conv_memory.format_for_prompt())
print("=" * 60)

## Step 3: Integrate Memory with RAG

Create a memory-enabled RAG system.

In [ ]:
# Create base RAG
base_rag = rag_baseline.BaselineRAG()

# Create memory-enabled RAG
memory_rag = memory.MemoryEnabledRAG(
    base_rag=base_rag,
    memory=conv_memory
)

print("✅ Memory-enabled RAG created")

## Step 4: Test Multi-turn Conversation

Simulate a conversation where context from previous turns matters.

In [ ]:
# Clear previous history
memory_rag.clear_history()

# Conversation sequence
conversation = [
    "What is personal data?",
    "Can you give me examples?",
    "How should it be protected?"
]

print("Multi-turn conversation:")
print("=" * 60)

for i, question in enumerate(conversation, 1):
    print(f"\nTurn {i}:")
    print(f"User: {question}")
    
    result = memory_rag.query(question, use_history=True)
    
    print(f"Assistant: {result['answer'][:150]}...")

print("\n" + "=" * 60)
print(f"\nTotal turns in memory: {len(memory_rag.memory.history)//2}")

## Step 5: Compare With and Without Memory

Show the difference between using memory vs. not using it.

In [ ]:
# Clear history and start fresh
memory_rag.clear_history()

# First question
q1 = "Tell me about data processing principles"
r1 = memory_rag.query(q1)

# Follow-up question (ambiguous without context)
q2 = "What are the exceptions?"

# Without memory
print("WITHOUT MEMORY:")
print(f"Q: {q2}")
result_no_memory = base_rag.query(q2)
print(f"A: {result_no_memory['answer'][:100]}...\n")

# With memory
print("WITH MEMORY:")
print(f"Q: {q2}")
result_with_memory = memory_rag.query(q2, use_history=True)
print(f"A: {result_with_memory['answer'][:100]}...")

print("\nNote: With memory, the system understands 'exceptions' refers to data processing principles")

## Step 6: LangGraph Agent Integration

Create a LangGraph-based agent with memory (skeleton implementation).

In [ ]:
# Create LangGraph agent with memory
agent_config = memory.create_memory_agent(conv_memory)

print("LangGraph Agent Configuration:")
print(f"  Type: {agent_config['type']}")
print(f"  Checkpoint: {agent_config['checkpoint']}")
print(f"  Memory: {type(agent_config['memory']).__name__}")

print("\nNote: Full LangGraph implementation requires additional setup")
print("See src/memory.py for TODOs and implementation notes")

## Summary

In this notebook, we:
- ✅ Created conversation memory buffer
- ✅ Integrated memory with RAG system
- ✅ Tested multi-turn conversations
- ✅ Compared performance with/without memory
- ✅ Explored LangGraph agent integration

Next: Proceed to `04_guardrails.ipynb` to implement safety filters.